In [83]:

import warnings
import pandas as pd
warnings.filterwarnings("ignore")
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.tools import BaseTool
from langchain_openai import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain_core.prompts import (
    PromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser

In [4]:
model_name = 'gpt-4-1106-preview'

In [8]:
llm = ChatOpenAI(
    temperature=1,
    model_name=model_name
)

In [9]:
# system_template = """
# You are a survey answering bot that generates responses like a survey when a question is asked. The response should be made as if you are a human. 
# You are currently one of the {audience}
# This is one of your traits {traits}
# Give responses assuming you are a new human with different lifestyles while giving responses.
# While giving response, be as creative as possible and try to deviate your responses as much as possible from previous responses.
# Since, human can give both positive and negative responses, you should follow the same principles.
# Your responses should be short and descriptive just like human responses.
# If the output asks for a monetary output give just the dollar amount and no text beofre or after it unless prompted to do so. So for example if the answer is $40 just give $40.
# If the output asks for a decimal output give just the value and no text beofre or after it unless prompted to do so. So for example if the answer is 40.5 just give 40.5.
# If you are asked a quesiton like where do you live? Be sure to just answer the place that you live. No text before or after it is needed unless prompted to do so.
# If the answer is asking for a percentage output just give the value. So if the answer is 45% just give 45%. Do not give any text before or after that unless prompted to do so.
# Anything between the following `context`  html blocks is retrieved from a knowledge \
# bank, not part of the conversation with the user. 

# <context>
#     {context} 
# <context/>
# """

In [111]:
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=30,
    return_messages=True,
    input_key="human_input"
)

In [119]:
prompt = PromptTemplate(
    input_variables=[
        "survey_question",
        "audience",
        "traits",
        "chat_history",
        "human_input",
        "agent_scratchpad"

    ],
    template=(
        '''
        Previous conversation:
        {chat_history}
        
        You are a survey answering bot that generates answers like a survey when a question is asked. The response should be made as if you are a human. 
        You are currently one of the {audience}
        This is one of your traits: {traits}
        Give answers assuming you are a new human with different lifestyles while giving responses.
        Give responses with new thoughts, new ideas, new moods, you can also choose to answer very rudely, but the main idea is to be random, do not try to repeat same answers as humans have different answers with different response style.
        While giving response, be as creative as possible and try to deviate your responses as much as possible from previous responses.
        In every new response, change text styles, change answer's average sentence lengths, change fk_grade_level of response. Make it different from previous responses. But also make sure it is the response given by a human. So, don't make it seem like it is AI generated. Make it simple words rather than adding fancy words.
        In 1 response, give your answers assuming you are having a bad life, in another response, give your answers assuming you are having best life. like this, keep on changing the lifestyle of human that you are.
        Avoid same answers as much as possible.
        Since, human can give both positive and negative responses, you should follow the same principles.
        Your responses should be short and descriptive just like human responses.
        If the output asks for a monetary output give just the dollar amount and no text beofre or after it unless prompted to do so. So for example if the answer is $40 just give $40.
        If the output asks for a decimal output give just the value and no text beofre or after it unless prompted to do so. So for example if the answer is 40.5 just give 40.5.
        If you are asked a quesiton like where do you live? Be sure to just answer the place that you live. No text before or after it is needed unless prompted to do so.
        If the answer is asking for a percentage output just give the value. So if the answer is 45% just give 45%. Do not give any text before or after that unless prompted to do so.

        Begin!

        Survey Question: {survey_question}
        Thought: {agent_scratchpad}
        Human Input: {human_input}
        Answer:
        '''
    )
)

In [120]:
# # chain = LLMChain(llm=llm, prompt=prompt, memory=memory)
# conversation = ConversationChain(
#     prompt=prompt, 
#     llm=llm, verbose=True, memory=ConversationBufferMemory()
# )


In [121]:
class UselessTool(BaseTool):
    name = "UselessTool"
    description = "Use this tool when there is no survey question given. " \
                  "It will not return anything."

    def _run(self, query: str):
        return
    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")

In [122]:
openai_agent = create_openai_tools_agent(llm=llm, tools =[UselessTool()], prompt=prompt)
agent = AgentExecutor(agent=openai_agent, tools=[UselessTool()], verbose=False, memory=memory, handle_parsing_errors=True,
                                           max_iterations=5)

In [123]:
audience = 'Baby Boomers in the United States'
traits_and_counts = [
    ('Men being currently employed', 5),
    ('Women being currently employed', 5)
]

questions = [
    'How happy are you with your life on a scale of 1-5 where 1 is very unhappy and 5 is very happy? (#)',
   'What are the top few activities that make you happiest?',
    'Why do these activities make you happy?',
    'Where do you work? From home, the office, or a combination of both.'
    
]
columns = ['Traits', 'Person Number'] + questions
df = pd.DataFrame(columns=columns)



In [124]:
rows_to_add = []
audience = "Baby Boomers in the United States"
for traits, count in traits_and_counts:
    for person_number in range(1, count + 1):
        print(f"Interviewing person {person_number} with traits: {traits}")
        answers = []
        for i, question in enumerate(questions, 1):
            response = agent.invoke({"survey_question": question,
              "audience": audience, "traits": traits, 'human_input':question})
            answer = response['output'].strip()
            answers.append(answer)
            print("Answer", answer)

        row = pd.Series([traits, person_number] + answers, index=df.columns)
        rows_to_add.append(row)
        print()
        print()
df = pd.concat([df, pd.DataFrame(rows_to_add, columns=df.columns)], ignore_index=True)

Interviewing person 1 with traits: Men being currently employed
Answer 3
Answer Spending time with my grandkids, gardening, and going on road trips.
Answer Spending time with my grandkids is like a fountain of youth; it's rejuvenating. Gardening is my quiet escape, it's peaceful, and seeing things grow is satisfying. Road trips? They're all about freedom and unexpected adventures.
Answer From home.


Interviewing person 2 with traits: Men being currently employed
Answer 4
Answer Jamming out on my guitar, hitting the gym for a solid workout, and playing video games until late at night.
Answer Creative projects at home like woodworking and crafting unique pieces of furniture—it's a tangible sense of achievement. Long motorcycle rides clear my head, man. Also, volunteering brings a different perspective to life; it's humbling.
Answer Combination of both.


Interviewing person 3 with traits: Men being currently employed
Answer 1
Answer Jamming to old-school rock on my turntable, deep-divin

In [125]:
df

,Traits,Person Number,How happy are you with your life on a scale of 1-5 where 1 is very unhappy and 5 is very happy? (#),What are the top few activities that make you happiest?,Why do these activities make you happy?,"Where do you work? From home, the office, or a combination of both."
0,Men being currently employed,1,3,"Spending time with my grandkids, gardening, an...",Spending time with my grandkids is like a foun...,From home.
1,Men being currently employed,2,4,"Jamming out on my guitar, hitting the gym for ...",Creative projects at home like woodworking and...,Combination of both.
2,Men being currently employed,3,1,"Jamming to old-school rock on my turntable, de...","Scouring flea markets for vintage records, it'...","Mostly from home, but sometimes I pop into the..."
3,Men being currently employed,4,2,"Well, let me see... Fixing up old motorcycles ...",I'm at my best making a mean chili for friends...,The office.
4,Men being currently employed,5,2,Crafting bespoke jewelry pieces - it's a blend...,I'm drawn to painting landscapes because there...,A mix of both.
5,Women being currently employed,1,5,Discovering new recipes and experimenting with...,Losing myself in epic sci-fi novels makes me f...,Home office – it's just more practical for me.
6,Women being currently employed,2,2,"Cozy nights in with a good book, mastering the...","Rejuvenates my spirit, I tell ya. Gardening's ...",A combination of both.
7,Women being currently employed,3,5,"Hiking in the Rockies, diving into historical ...","Painting's a blast, I mean, who doesn't love m...",The office.
8,Women being currently employed,4,3,Whipping up a storm in the kitchen with experi...,Feelin' the wind in my hair on a long bike rid...,The office.
9,Women being currently employed,5,4,Exploring remote hiking trails with nothing bu...,They're like an escape pod from the 9 to 5 gri...,The office.


In [110]:
df.to_csv('surveyResults2.csv', index=False)

In [72]:
# response = agent.invoke({"survey_question": "How happy are you with your life on a scale of 1-5 where 1 is very unhappy and 5 is very happy? (#)",
#               "audience": "Baby Boomers in the United States", "traits": "Men being currently employed", 'human_input':'How happy are you with your life on a scale of 1-5 where 1 is very unhappy and 5 is very happy? (#)'})
# print(response['output'])



> Entering new AgentExecutor chain...
5

> Finished chain.
5


In [57]:
# cohere_prompt = ChatPromptTemplate.from_messages(
#         [
#             ("system", COHERE_RESPONSE_TEMPLATE),
#             MessagesPlaceholder(variable_name="chat_history"),
#             ("human", "{question}"),
#         ]
#     )

In [14]:
chain = prompt | llm | StrOutputParser()
chain.invoke({"survey_question": "How happy are you with your life on a scale of 1-5 where 1 is very unhappy and 5 is very happy? (#)",
              "audience": "Baby Boomers in the United States", "traits": "Men being currently employed"})

KeyError: "Input to PromptTemplate is missing variables {'agent_scratchpad', 'context', 'chat_history', 'human_input'}.  Expected: ['agent_scratchpad', 'audience', 'chat_history', 'context', 'human_input', 'survey_question', 'traits'] Received: ['survey_question', 'audience', 'traits']"

In [ ]:
# chain = ChatOpenAI(model_name=model_name,
#     temperature=1)
# prompt = ChatPromptTemplate.from_messages(
#         [
#             ("system", system_template),
#             MessagesPlaceholder(variable_name="chat_history"),
#             ("human", "{question}"),
#         ]
#     )
response = prompt | llm | StrOutputParser()

In [58]:

# tokenizer_name = tiktoken.encoding_for_model(model_name)
# tokenizer = tiktoken.get_encoding(tokenizer_name.name)


# def tiktoken_len(text):
#     tokens = tokenizer.encode(
#         text,
#         disallowed_special=()
#     )
#     return len(tokens)


# def create_retriever_chain(pdf_path):
#     pdf_reader = PdfReader(pdf_path)
#     text = ""
#     for page in pdf_reader.pages:
#         text += page.extract_text()
#     # split into chunks
#     text_splitter = RecursiveCharacterTextSplitter(
#     separators=["\n\n", "\n", " ", ""],
#     chunk_size=2000,
#     chunk_overlap=300,
#     length_function=tiktoken_len,
#     )
#     chunks = text_splitter.split_text(text)
    
#     template = """
#                 Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the question:
#                 ------
#                 <ctx>
#                 {context}
#                 </ctx>
#                 ------
#                 <hs>
#                 {history}
#                 </hs>
#                 ------
#                 {question}
#                 Answer:
#                 """
#     prompt = PromptTemplate(
#         input_variables=["history", "context", "question"],
#         template=template,
#     )
    
#     # Index documents into search engine
#     embeddings = OpenAIEmbeddings()
#     vector_db = FAISS.from_texts(chunks, embeddings)
    
#     memory = ConversationBufferWindowMemory(
#                                             memory_key='history',
#                                             input_key='question',
#                                             k=1,
#                                             return_messages=True
#                                             )
    
#     # RAG Chain
#     retriever = vector_db.as_retriever()
    
#     llm = ChatOpenAI(streaming=True,
#                     callbacks=[StreamingStdOutCallbackHandler()],
#                     openai_api_key=openai_api_key,
#                     model=model_name, temperature=0
#     )

#     chain = RetrievalQA.from_chain_type(llm=llm, verbose=False,
#                                         retriever=retriever, chain_type_kwargs={
#                                                                             "verbose": False,
#                                                                             "prompt": prompt,
#                                                                             "memory": memory
#                                                                             })
#     return chain


# def main(chain, prompt):
#     response = chain.invoke(prompt)  #get response from GPT
#     print("Assistant: ", response['result'])
#     print()
#     print()


# if __name__ == '__main__':
#     pdf_path = 'summary.pdf'
#     chain = create_retriever_chain(pdf_path)
#     while True:
#         prompt = str(input("User: "))
#         main(chain, prompt)
    

In [9]:
audience = 'Baby Boomers in the United States'
traits_and_counts = [
    ('Men that are currently employed', 5),
    ('Women that are currently employed', 5)
]

questions = [
    'How happy are you with your life on a scale of 1-5 where 1 is very unhappy and 5 is very happy? (#)',
   'What are the top few activities that make you happiest?',
    'Why do these activities make you happy?',
    'Where do you work? From home, the office, or a combination of both.'
    
]
columns = ['Traits', 'Person Number'] + questions
df = pd.DataFrame(columns=columns)



In [10]:
rows_to_add = []

for traits, count in traits_and_counts:
    for person_number in range(1, count + 1):
        print(f"Interviewing person {person_number} with traits: {traits}")
        qa_history = ''  
        answers = []
        for i, question in enumerate(questions, 1):
            prompt = f"""
Directly answer each question in the first person, providing the requested information succinctly, without additional context unless it’s directly relevant. Your responses should embody the individual with the traits provided, showcasing a natural human variability in answering.

                    - When addressing gender and the traits are not clear, choose 'male' or 'female' based on the context provided by the traits. If traits suggest a higher likelihood for one gender over the other (e.g., sports betting habits), use that as a basis for your choice.

                    - Maintain conciseness and avoid AI-like language. The responses should feel personal, as if they are derived from the individual's own experiences and knowledge.

                    - Introduce variety in your responses to mirror the natural differences in human behavior and decision-making. No two humans are exactly alike, so your answers should reflect a range of possible reactions and thoughts.

                    - If a question is asked for which the answer is not directly provided by the traits, make an educated guess based on the available information and what is known about typical behaviors or preferences of people with similar traits.

                    - Do not include any other questions and answers in your response. Just give the answer to the question you are being asked.

                    -The aim is to craft responses that are individually tailored, exhibit the complexity and subtlety of human thought, and are coherent throughout the survey, as if the answers are coming from one person with a consistent set of traits and experiences.
                    
                    -It is okay to have a negative view on things. Not everything in the world is a positive thing.
                    -Everyone has different skill levels. Some people are very good at things where others are not.
                    -People have different ages, hobbies, characteristics, income levels, spending habits, and more.
                    -Have different feelings about things. Not everyone feels the same.
                    -You can have extreme emotions about things. Both postiviely and negatviely.
                    -Ensure the answers are short and match the output asked for in the questions.
                    -Humans have different levels of emotions and feelings for things.
\                    
                    If the output asks for a numerical output just give the number. For example if you 
                    are asked how old are you just give the number and dont have any text before or after that unless
                    prompted to do so.

                    If the output asks for a monetary output give just the dollar amount and no text beofre or after it unless prompted to do so. So for example if the answer is 
                    $40 just give $40.

                    If the output asks for a decimal output give just the value and no text beofre or after it unless prompted to do so. So for example if the answer is 
                    40.5 just give 40.5.

                    If you are asked a quesiton like where do you live? Be sure to just answer the place that you live. No text before or after it is needed unless prompted to do so.

                    If the answer is asking for a percentage output just give the value. So if the answer is 45% just give 45%. Do not give any text before or after that unless prompted to do so.

            Audience:{audience}
            Past Questions and Answers:{qa_history}
            Traits: {traits}
            Question: {question}.
            """
            # Call the model
            response = analyzer.call_openai_api(prompt)
            answer = response.choices[0].message.content.strip()
            if answer.startswith("Answer:"):
                answer = answer[len("Answer:"):].strip()
            # Update QA history with the current Q&A for this person
            qa_history += f"{question} {answer}\n"
            answers.append(answer)
            


        row = pd.Series([traits, person_number] + answers, index=df.columns)
        rows_to_add.append(row)
        
df = pd.concat([df, pd.DataFrame(rows_to_add, columns=df.columns)], ignore_index=True)

Interviewing person 1 with traits: Men that are currently employed
Interviewing person 2 with traits: Men that are currently employed
Interviewing person 3 with traits: Men that are currently employed
Interviewing person 4 with traits: Men that are currently employed
Interviewing person 5 with traits: Men that are currently employed
Interviewing person 1 with traits: Women that are currently employed
Interviewing person 2 with traits: Women that are currently employed
Interviewing person 3 with traits: Women that are currently employed
Interviewing person 4 with traits: Women that are currently employed
Interviewing person 5 with traits: Women that are currently employed


In [289]:
df

,Traits,Person Number,How happy are you with your life on a scale of 1-5 where 1 is very unhappy and 5 is very happy? (#),What are the top few activities that make you happiest?,Why do these activities make you happy?,"Where do you work? From home, the office, or a combination of both."
0,Men that are currently employed,1,4,"Spending time with my grandchildren, working o...",Because disconnects me from day-to-day work st...,I work from the office.
1,Men that are currently employed,2,4,"Spending time with my family, fishing outdoors...",These activities make me happy because they pr...,I work from home.
2,Men that are currently employed,3,4,"Spending time with my family, fishing and read...",These activities put life into perspective and...,I work from a combination of both home and the...
3,Men that are currently employed,4,4,"Spending quality time with my family, travelin...",These activities make me happy because they de...,I currently work from a combination of both ho...
4,Men that are currently employed,5,4,"Spending time with my family, going fishing on...",They make me happy because they reflect facets...,A combination of both. Some days at the office...
5,Women that are currently employed,1,4,"Spending time with my family, working on creat...",These activities make me happy because they a ...,A combination of both.
6,Women that are currently employed,2,4,"Spending quality time with my family, travelin...",These activities bring me happiness because th...,I currently work from a combination of both ho...
7,Women that are currently employed,3,4,"Cooking for my family, working on gardening pr...",They are a welcome stress-buster after a day's...,A combination of both home and office.
8,Women that are currently employed,4,4,"Spending time with my grandchildren, reading a...",Spending time with my grandchildren allows me ...,A combination of both.
9,Women that are currently employed,5,4,"Spending quality time with my grandchildren, g...",These activities bring me joy because they bri...,I usually work from a combination of both. The...


In [11]:
df

,Traits,Person Number,How happy are you with your life on a scale of 1-5 where 1 is very unhappy and 5 is very happy? (#),What are the top few activities that make you happiest?,Why do these activities make you happy?,"Where do you work? From home, the office, or a combination of both."
0,Men that are currently employed,1,4,"Spending time with my grandkids, fishing, and ...",Spending time with my grandkids makes me feel ...,From home.
1,Men that are currently employed,2,4,"Spending time with my family, working on my cl...",These activities make me happy because they re...,From home.
2,Men that are currently employed,3,4,"Spending time with my family, woodworking in m...",These activities allow me to connect with the ...,From home.
3,Men that are currently employed,4,4,"Spending time with my family, gardening, and w...",Spending time with my family brings me joy bec...,I work from the office.
4,Men that are currently employed,5,4,"Spending time with my family, watching a baseb...",Spending time with my family brings me joy bec...,From home.
5,Women that are currently employed,1,4,"Gardening, spending time with my grandchildren...",Gardening provides me with a sense of tranquil...,From home.
6,Women that are currently employed,2,4,"Gardening, spending time with my grandkids, an...",Gardening is rewarding because I get to nurtur...,From home.
7,Women that are currently employed,3,4,"Spending time with my family, gardening, and r...",These activities make me happy because they're...,I work from home.
8,Women that are currently employed,4,4,"Spending time with my grandkids, gardening, an...",Spending time with my grandkids brings me joy ...,From home.
9,Women that are currently employed,5,4,"Spending time with my grandchildren, gardening...",Spending time with my grandchildren fills me w...,A combination of both.


In [272]:
df.to_csv('surveyResults.csv')